In [1]:
import pandas as pd
import numpy as np

def read_data(filename):
    df = pd.read_csv(filename, index_col=0)
    df['same_security'] = df['same_security'].astype(int)
    
    return df

In [6]:
train_df = read_data("dataset/Phase 1 - similarityCompetition/train.csv")

In [9]:
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

stop_words = set(stopwords.words('english'))
puncuation = set(string.punctuation)
puncuation.add("-")
puncuation.add("'")
puncuation.add("\"")
snowball = SnowballStemmer('english')
lemma = WordNetLemmatizer()

def clean(string):
    stop_free = " ".join([i for i in word_tokenize(string.lower()) if i not in stop_words])
    punc_free = "".join([ch for ch in stop_free if ch not in puncuation])
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    snowed = " ".join(snowball.stem(word) for word in normalized.split())
    
    return snowed

In [37]:
cleaned_desc_a = train_df['description_x'].apply(lambda x: clean(x))
cleaned_desc_b = train_df['description_y'].apply(lambda x: clean(x))

In [52]:
t = list(train_df['same_security']) + list(train_df['same_security'])
len(pd.concat((cleaned_desc_a, cleaned_desc_b), ignore_index=True))

4284

In [53]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in self.doc_list.items():
            yield TaggedDocument(words=doc.split(), tags=[self.labels_list[idx]])
 
data = list(train_df['same_security']) + list(train_df['same_security'])
sentences = TaggedDocumentIterator(pd.concat((cleaned_desc_a, cleaned_desc_b), ignore_index=True), data)

model = Doc2Vec(vector_size=100, window=10, min_count=5, workers=11,alpha=0.025, epochs=20)
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)

/Users/michaelbecker/anaconda/envs/deeplearning/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


In [61]:
one = TaggedDocument(words=cleaned_desc_a.iloc[0].split(), tags=0)
two = TaggedDocument(words=cleaned_desc_b.iloc[0].split(), tags=1)

In [71]:
new_doc_vec = model.infer_vector(cleaned_desc_b.iloc[1].split(), steps=50, alpha=0.25)
model.docvecs.most_similar(positive=[new_doc_vec])

[(1, 0.5910154581069946), (0, 0.45649105310440063)]